# Load libraries

In [ ]:
import os
import numpy as np
import ase
from ase.visualize import view
from abtem.visualize import show_atoms
import abtem
from abtem.potentials import Potential
import hyperspy.api as hs
import scipy
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.signal import get_window
import cv2
from scipy import ndimage

# Change default figure size

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]

# Loop through simulation cells and simulate STEM images

In [ ]:
folder_path = './' # specifiy folder where geometry files are located
cells = [os.path.join(folder_path, _) for _ in os.listdir(folder_path) if _.endswith('.vasp')]
print(cells)

## Set up probe for convolution

In [ ]:
# sampling
sampling = 0.12

In [ ]:
from abtem import *
from abtem.waves import Probe

#Set up the electron probe
probe = Probe(energy=300e3, sampling=sampling, extent=2, semiangle_cutoff=24, focal_spread=60, defocus=0)
    
positions = [(1, 1)]
waves = probe.build(positions)
#wave = waves[0]
#wave.show()
    
probe_image = np.abs(waves.array) ** 2
plt.imshow(probe_image)
plt.colorbar()
plt.show()

In [ ]:
from ase.io import read
from abtem import *
from abtem.waves import Probe
from skimage.filters import gaussian
import cv2
import time
import h5py

start = time.time()
plot_for_testing = 1
save_to_file = 1

for cell in cells:
    
    # Read simulation cell and atom positions from file
    atoms = read(cell, format="vasp")

    # Determine slice thickness
    eps = 0.001 
    atoms_pos = atoms.get_positions()
    atoms_z = atoms_pos[:,2]
    slice_thickness = np.min(atoms_z[atoms_z>eps])

    # Compute the projected potential
    projpot = Potential(atoms, sampling=.12, slice_thickness=slice_thickness, projection='finite', parametrization='kirkland')
    precalc_pot = projpot.build(pbar=True)
    precalc_pot = np.sum(precalc_pot.array, axis=0)
    
    # Convolve with probe
    conv_stem = cv2.filter2D(precalc_pot, -1, probe_image)
    
    if plot_for_testing == 1:
    
        #Plot summed projected potential and convolved STEM image for testing
        fig, axes = plt.subplots(nrows=1, ncols=1)
        fig.tight_layout()

        plt.subplot(1, 2, 1)
        plt.imshow(precalc_pot, cmap='plasma')
        plt.title('Summed projected potential')
        plt.axis('off')
        #plt.clim(0, 0.4)
        plt.colorbar()

        plt.subplot(1, 2, 2)
        plt.imshow(conv_stem, cmap='plasma')
        plt.title('Convolved STEM image')
        plt.axis('off')
        #plt.clim(0, 0.5)
        plt.colorbar()
        plt.show()
        
    if save_to_file == 1:
        
        # Save to hdf5 file
        with h5py.File(cell[:-5] + '_convolved_STEM_image.hdf5', 'w') as convolved_stem_image:
            group_conv_stem = convolved_stem_image.create_group('convolved_stem_image')
            group_conv_stem.create_dataset('stem_probe', data=probe_image)
            group_conv_stem.create_dataset('precalc_pot', data=precalc_pot)
            group_conv_stem.create_dataset('conv_stem', data=conv_stem)

            # Create attributes
            group_conv_stem.attrs['Data description'] = ['Here each convolved STEM image is obtained by convolving the STEM probe with the summer projected potential.']

            ## Close hdf5 file(s)  
            #file.close()
    
    #print ('time cost: %6.2f[s]' %(time.time()-start,))